In [8]:
import cv2
import subprocess
import numpy as np
import glob
import time
import shlex
from loguru import logger
from dice_combinations import DICE_COMBINATIONS


FPS = 30
HEIGHT = 540
WIDTH = 960
RTMP_URL = "rtmp://vid.heeraexch.com:1935/live/sicbo"

common_image_paths = glob.glob(
    "/Users/rkhilnan/Desktop/awone_docs/sicbo/common_segment/*.jpg"
)
common_image_paths = sorted(
    common_image_paths, key=lambda x: int(x.split("/")[-1].split(".")[0])
)
# common_images = [cv2.imread(impath) for impath in common_image_paths]

combo_paths = {}
for combo in DICE_COMBINATIONS:
    combo_str = "".join([str(digit) for digit in combo])
    paths = glob.glob(f"/Users/rkhilnan/Desktop/awone_docs/sicbo_new/{combo_str}/*.jpg")
    paths = sorted(paths, key=lambda x: int(x.split("/")[-1].split(".")[0]))
    combo_paths[combo_str] = paths


# command and params for ffmpeg
command = ['ffmpeg',
           '-y',
           '-r', str(FPS),
           '-f', 'rawvideo',
           '-vcodec', 'rawvideo',
           '-pix_fmt', 'bgr24',
           '-s', "{}x{}".format(WIDTH, HEIGHT),
           '-i', '-',
           '-c:v', 'libx264',
           '-pix_fmt', 'yuv420p',
           '-preset', 'ultrafast',
           '-f', 'flv',
           RTMP_URL]

In [9]:
p = subprocess.Popen(command, stdin=subprocess.PIPE)
font = cv2.FONT_HERSHEY_SIMPLEX
k = 0
m = 0
print(len(common_image_paths))
while True:
    m += 1
    try:
        ts = time.time()
        print("ts:", ts)
        for ix, impath in enumerate(common_image_paths):
            im = cv2.imread(impath)
            k += 1

            # Call start bets event here and stop bets after X secs
            if ix == 0:
                round_start_script = f"python3 start_event.py --ts {str(ts)}"
                shlex_bet = shlex.split(round_start_script)
                subprocess.Popen(shlex_bet, start_new_session=True)
            time.sleep(1 / 41)
            cv2.putText(
                im, f"Common: {k}", (10, 100), font, 3, (0, 255, 0), 2, cv2.LINE_AA
            )
            try:
                p.stdin.write(im.tobytes())
            except:
                time.sleep(0.3)
                im = cv2.imread(impath)
                p.stdin.write(im.tobytes())
            if ix == (len(common_image_paths) - 100):
                bet_stop_script = f"python3 nomorebets.py --ts {str(ts)}"
                shlex_bet = shlex.split(bet_stop_script)
                subprocess.Popen(shlex_bet, start_new_session=True)
                generate_numbers_script = f"python3 gen_frames.py --ts {str(ts)}"
                shlex_bet = shlex.split(generate_numbers_script)
                subprocess.Popen(shlex_bet, start_new_session=True)

        dice_numbers = open("sicbo_dice_numbers.txt", "r").read()
        dice1 = dice_numbers.split(",")[0].strip()
        dice2 = dice_numbers.split(",")[1].strip()
        dice3 = dice_numbers.split(",")[2].strip()
        n_paths = combo_paths[dice1 + dice2 + dice3]
        for ix, impath in enumerate(n_paths):
            k += 1
            # Call start bets event here and stop bets after X secs
            im = cv2.imread(impath)
            time.sleep(1 / 41)
            cv2.putText(im, str(k), (10, 100), font, 3, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(im, str(m), (10, 450), font, 3, (0, 255, 0), 2, cv2.LINE_AA)
            try:
                p.stdin.write(im.tobytes())
            except:
                time.sleep(0.3)
                im = cv2.imread(impath)
                p.stdin.write(im.tobytes())
            if ix == len(n_paths) - 100:
                cmd3 = f"python3 roundend.py --ts {str(ts)}"
                cmds3 = shlex.split(cmd3)
                p5 = subprocess.Popen(cmds3, start_new_session=True)
    except KeyboardInterrupt:
        p.stdin.close()


138
ts: 1646289308.054325


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox --

ts: 1646289316.085748


2022-03-03 12:05:18.643 | INFO     | __main__:<module>:22 - File opened!.0kbits/s speed=0.745x    
Traceback (most recent call last):  6802kB time=00:00:36.86 bitrate=1511.5kbits/s speed=0.79x     
  File "/Users/rkhilnan/Desktop/projects/sicbo/roundend.py", line 13, in <module>
    dice_numbers = open("sicbo_numbers.txt", "r").read()
FileNotFoundError: [Errno 2] No such file or directory: 'sicbo_numbers.txt'


ts: 1646289360.551382


...

ts: 1646289582.401572


2022-03-03 12:09:44.779 | INFO     | __main__:<module>:22 - File opened!.5kbits/s speed=0.808x    


ts: 1646289592.124924


[flv @ 0x13de05560] Failed to update header with correct duration.
[flv @ 0x13de05560] Failed to update header with correct filesize.
frame= 6865 fps= 24 q=-1.0 Lsize=   43698kB time=00:03:48.80 bitrate=1564.6kbits/s speed=0.812x    
video:43563kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.308440%
[libx264 @ 0x13de062c0] frame I:28    Avg QP:15.61  size: 96248
[libx264 @ 0x13de062c0] frame P:6837  Avg QP:18.37  size:  6130
[libx264 @ 0x13de062c0] mb I  I16..4: 100.0%  0.0%  0.0%
[libx264 @ 0x13de062c0] mb P  I16..4:  2.5%  0.0%  0.0%  P16..4: 31.0%  0.0%  0.0%  0.0%  0.0%    skip:66.5%
[libx264 @ 0x13de062c0] coded y,uvDC,uvAC intra: 35.7% 66.5% 29.8% inter: 9.9% 20.9% 3.1%
[libx264 @ 0x13de062c0] i16 v,h,dc,p: 40% 23% 23% 14%
[libx264 @ 0x13de062c0] i8c dc,h,v,p: 37% 25% 29%  9%
[libx264 @ 0x13de062c0] kb/s:1559.51
Exiting normally, received signal 2.


ValueError: write to closed file